In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import hpvsim as hpv
import time as t 
import random as rn
import math as m 
import seaborn as sb
import matplotlib.cm as cm
import ast
import matplotlib.patches as patches

HPVsim 2.0.0 (2023-11-29) — © 2023 by IDM


In [2]:
cmap = plt.get_cmap('viridis')
colors = [cmap(i) for i in np.linspace(0, 1, 3)]

Dictionnaire des paramètres

In [3]:
#-------------------------------------------------------------
# Simulation parameters
#-------------------------------------------------------------
n_agents = 50000 # Anas analysis
location = 'tunisia' 
start = 1990 # calibration reasons
end = 2090 
burnin = 20
dt = 0.25
dt_demog = 1
network = 'default'
ms_agent_ratio = 1
rel_birth = 1
rel_death = 1
sex_ratio = 0.5
rel_init_prev =1
rand_seed = 1
verbose = 0.25
use_migration = False

#-------------------------------------------------------------
# Initialization parameters
#-------------------------------------------------------------
init_hpv_prev = {
                'age_brackets': [30, 40, 50, 150], 
                'm': [0.124, 0.07, 0.064, 0.081], 
                'f': [0.124, 0.07, 0.064, 0.081]
                }
init_hpv_dist = {
                'hpv31': 0.019,
                'hpv16': 0.011,
                'hpv59': 0.0125,
                'hpv52': 0.012,
                'hpv18': 0.011,
                'hpv35': 0.0055,
                'hpv58': 0.0065,
                'hpv56': 0.0045,
                'hpv39': 0.0045, 
                'lr': 0.9135
                }


#-------------------------------------------------------------
# Sexual Behavior parameters
#-------------------------------------------------------------
cross_layer_options = [
    dict(
        f=0.05, # default value in HPVsim
        m=0.3, # default value in HPVsim
    ),
    dict(
        f=0.1, 
        m=0.25, 
    ),
]
cross_layer_option = cross_layer_options[1]  # change here for cross-layer mixing choice (0: default, 1: results of calibration for Tunisia)

f_cross_layer = cross_layer_option['f']
m_cross_layer = cross_layer_option['m']

debut = dict( 
    f = dict(dist='normal', par1=23, par2=3), # source: GTBC + INS
    m = dict(dist='normal', par1=22, par2=4),
)

dur_pship_options = [
    dict(
            m=dict(dist='neg_binomial', par1=80, par2=3), # This gives: mar_dur = {'0-5y': 0.015, '5-10y': 0.025, '10-20y':0.06, '20-50y':0.25, '50+':0.65}
            c=dict(dist='lognormal', par1=1, par2=2), # This gives: cas_dur = {'0-3m': 0.33, '3-6m': 0.22, '6-12m': 0.2, '1-2y':0.15, '2-5y':0.1}
    ),
    
    dict( # change casual ? 
            m=dict(dist='lognormal', par1=20, par2=26), 
            c=dict(dist='lognormal', par1=1, par2=2), # This gives: cas_dur = {'0-3m': 0.33, '3-6m': 0.22, '6-12m': 0.2, '1-2y':0.15, '2-5y':0.1}
    ),
]
dur_pship = dur_pship_options[1]  # change here for duration of partnership choice (0: default, 1: our assumption for Tunisia)

#defaults (do not change)
m_partners = dict(
    m=dict(dist='poisson1', par1=0.01), # Everyone in this layer has one marital partner; this captures *additional* marital partners. If using a poisson distribution, par1 is roughly equal to the proportion of people with >1 spouse
    c=dict(dist='poisson1', par1=0.5)
) # If using a poisson distribution, par1 is roughly equal to the proportion of people with >1 casual partner within a single time step

f_partners = dict(
    m=dict(dist="poisson1", par1=0.01),
    c=dict(dist='poisson', par1=1), # Defaults: {'0': 0.36, '1': 0.37, '2': 0.19, '3': 0.06, '4+':0.02}
)

layer_probs_options = [
    dict( #all default
            m=np.array([
                [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
                [ 0,  0,  0.01,   0.5,  0.5,  0.5,  0.5,  0.5,   0.5,   0.5,   0.5,  0.3,  0.2,  0.1, 0.05, 0.01], # Share of females of each age who are actively seeking marriage if underpartnered
                [ 0,  0,  0.01,   0.2,  0.3,  0.5,  0.5,  0.5,   0.5,   0.5,   0.5,  0.3,  0.2,  0.1, 0.05, 0.01]] # Share of males of each age who are actively seeking marriage if underpartnered
                # [ 1,  1,     1,     1,    1,    1,    1,    1,     1,     1,     1,    1,    1,    1,    1,    1]] # Share of males of each age who are actively seeking marriage if underpartnered
            ),
            c=np.array([
                [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
                [ 0,  0,   0.2,   0.6,  0.8,  0.6,  0.4,  0.4,   0.4,   0.1,  0.02, 0.02, 0.02, 0.02, 0.02, 0.02], # Share of females of each age actively seeking casual relationships if underpartnered
                [ 0,  0,   0.2,   0.4,  0.4,  0.4,  0.4,  0.6,   0.8,   0.6,   0.2,  0.1, 0.05, 0.02, 0.02, 0.02]] # Share of males of each age actively seeking casual relationships if underpartnered
            )
    ),
    dict(   # marital INS, casual default
        m=np.array([  # source: https://books.openedition.org/ined/18618?lang=en     
            [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
            [ 0,  0,  0.01,   0.1,  0.4,  0.7,  0.8,  0.7,   0.7,   0.6,   0.6,  0.3,  0.2,  0.1, 0.05, 0.01], # Females actively seeking marriage if underpartnered
            [ 0,  0,  0.01,   0.05,  0.1,  0.5,  0.5,  0.8,   0.8,   0.8,   0.7,  0.4,  0.2,  0.1, 0.05, 0.01]] # Males actively seeking marriage if underpartnered
        ),
        c=np.array([
                [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
                [ 0,  0,   0.2,   0.6,  0.8,  0.6,  0.4,  0.4,   0.4,   0.1,  0.02, 0.02, 0.02, 0.02, 0.02, 0.02], # Share of females of each age actively seeking casual relationships if underpartnered
                [ 0,  0,   0.2,   0.4,  0.4,  0.4,  0.4,  0.6,   0.8,   0.6,   0.2,  0.1, 0.05, 0.02, 0.02, 0.02]] # Share of males of each age actively seeking casual relationships if underpartnered
            )
    ),
    dict(   # marital INS, casual assumption
        m=np.array([  # source: https://books.openedition.org/ined/18618?lang=en     
            [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
            [ 0,  0,  0.01,   0.1,  0.4,  0.7,  0.8,  0.7,   0.7,   0.6,   0.6,  0.3,  0.2,  0.1, 0.05, 0.01], # Females actively seeking marriage if underpartnered
            [ 0,  0,  0.01,   0.05,  0.1,  0.5,  0.5,  0.8,   0.8,   0.8,   0.7,  0.4,  0.2,  0.1, 0.05, 0.01]] # Males actively seeking marriage if underpartnered
        ),
        c=np.array([  # assumption: casual relationships are less prevalent esp. with f .They are more common among younger people
            [ 0,  5,    10,    15,   20,   25,   30,   35,    40,    45,    50,   55,   60,   65,   70,   75],
            [ 0,  0,   0.01,  0.2,  0.4,  0.4,  0.2,  0.2,   0.2,   0.1,  0.02, 0.02, 0.02, 0.02, 0.02, 0.02], # Females actively seeking casual relationship if underpartnered
            [ 0,  0,   0.05,  0.5,  0.6,  0.7,  0.7,  0.7,   0.6,   0.4,   0.2,  0.1, 0.05, 0.01, 0.02, 0.02]] # Males actively seeking casual relationship if underpartnered
        )
    )
]

layer_probs = layer_probs_options[1]  # change here. 0: all default; 1: marital according INS and casual default; 2: marital INS and casual assumption. 

mixing_options = [
    np.array([  # source INS
        #       0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75   [F]
        [ 0,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 5,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [10,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15,    0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [20,    0,  0,  0,  0, .1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [25,    0,  0,  0, .4, .7, .4,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [30,    0,  0,  0,  1,  1,  1, .5, .1,  0,  0,  0,  0,  0,  0,  0,  0],
        [35,    0,  0,  0,  0, .6, .9,  1, .5, .1,  0,  0,  0,  0,  0,  0,  0],
        [40,    0,  0,  0, .2, .1, .3, .7,  1, .7, .1,  0,  0,  0,  0,  0,  0],
        [45,    0,  0,  0,  0,  0, .1, .3, .7,  1, .5, .1,  0,  0,  0,  0,  0],
        [50,    0,  0,  0,  0,  0,  0, .1, .2, .7,  1, .5, .1,  0,  0,  0,  0],
        [55,    0,  0,  0,  0,  0,  0,  0, .1, .2, .6,  1, .5, .1,  0,  0,  0],
        [60,    0,  0,  0,  0,  0,  0,  0,  0, .1, .2, .6,  1, .5, .1,  0,  0], 
        [65,    0,  0,  0,  0,  0,  0,  0,  0,  0, .1, .2, .6,  1, .5, .1,  0], 
        [70,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, .2, .6,  1, .5, .1],  
        [75,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0, .2, .6,  1,  1],
]),

    np.array([
        #       0,  5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75  [F]
        [ 0,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [ 5,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [10,    0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15,    0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [20,    0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [25,    0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [30,    0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [35,    0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0],
        [40,    0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0,  0],
        [45,    0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0,  0],
        [50,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
        [55,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0],
        [60,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,  0],
        [65,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0],
        [70,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0],
        [75,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1],
]),
]

mixing_option = mixing_options[0]
# change here for mixing matrix choice (0: Tunisian from INS marital data, 1: Anas extrapolation 2: Diagonal)
mixing = dict()
mixing['m'] = mixing_option
mixing['c'] = np.array([
            #       0,  5,  10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75
            [ 0,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [ 5,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [10,    0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [15,    0,  0,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [20,    0,  0,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0],
            [25,    0,  0, .5,  1,  1,  1,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0],
            [30,    0,  0,  0, .5,  1,  1,  1, .5,  0,  0,  0,  0,  0,  0,  0,  0],
            [35,    0,  0,  0, .5,  1,  1,  1,  1, .5,  0,  0,  0,  0,  0,  0,  0],
            [40,    0,  0,  0,  0, .5,  1,  1,  1,  1, .5,  0,  0,  0,  0,  0,  0],
            [45,    0,  0,  0,  0,  0,  1,  1,  1,  1,  1, .5,  0,  0,  0,  0,  0],
            [50,    0,  0,  0,  0,  0, .5,  1,  1,  1,  1,  1, .5,  0,  0,  0,  0],
            [55,    0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1, .5,  0,  0,  0],
            [60,    0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1, .5,  0,  0],
            [65,    0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1, .5,  0],
            [70,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1, .5],
            [75,    0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1],
        ])


#-------------------------------------------------------------
# Infection / Transmission parameters
#-------------------------------------------------------------

condoms = dict(
    m = 0.01, #default
    c = 0.18 # source GTBC
)
beta = 0.13 # calibration
transm2f = 3 # calibration
transf2m = 1 # calibration
eff_condoms = 0.7 # default https://www.nejm.org/doi/10.1056/NEJMoa053284?url_ver=Z39.88-2003&rfr_id=ori:rid:crossref.org&rfr_dat=cr_pub%20%200www.ncbi.nlm.nih.gov

#-------------------------------------------------------------
# Genotypess
#-------------------------------------------------------------
genotypes = ['hpv31', 'hpv16', 'hpv59', 'hpv52', 'hpv18', 'hpv35', 'hpv58', 'hpv56', 'hpv39', 'lr']      
n_genotypes =  10

#-------------------------------------------------------------
#-------------------------------------------------------------
#-------------------------------------------------------------
params= dict(
    n_agents = n_agents,
    location = location,
    dt = dt,
    dt_demog = dt_demog,
    network = network,
    start = start,
    end = end,
    burnin = burnin,
    debut = debut,
    m_partners = m_partners,
    f_partners = f_partners, 
    dur_pship = dur_pship,
    f_cross_layer = f_cross_layer,
    m_cross_layer = m_cross_layer,
    layer_probs = layer_probs,
    mixing = mixing,
    condoms = condoms,
    beta = beta,
    transm2f = transm2f,
    transf2m = transf2m,
    eff_condoms = eff_condoms,
    genotypes = genotypes,
    n_genotypes = n_genotypes,
    ms_agent_ratio = 1,
    rel_birth = 1,
    rel_death = 1,
    sex_ratio = 0.5,
    rel_init_prev =1,
    rand_seed = 1,
    verbose = 0.25,
    use_migration = False,
    init_hpv_prev = init_hpv_prev,
    init_hpv_dist = init_hpv_dist,
)



Simulation du scénario baseline

In [4]:
n=10
 

pap_test = hpv.routine_screening(
    product='pap',
    prob=0.01,
    age_range=[35, 60],
    start_year=2010,
    label='pap primary',
)
    
to_colposcopy = lambda sim: sim.get_intervention('pap primary').outcomes['abnormal']
colposcopy_test = hpv.routine_triage(
                product='colposcopy',
                prob = 0.9,
                eligibility=to_colposcopy,
                label = 'colposcopy secondary'
            )
        
    
hsils = lambda sim: sim.get_intervention('colposcopy secondary').outcomes['hsil']
excision = hpv.treat_num(
                prob = 0.9,
                product = 'excision',
                eligibility = hsils,
                label = 'excision'
            )
        
lsils = lambda sim: sim.get_intervention('colposcopy secondary').outcomes['lsil']
ablation = hpv.treat_num(
                prob = 0.9,
                product = 'ablation',
                eligibility = lsils,
                label = 'ablation'
            )


sim = hpv.Sim(params, interventions=[pap_test, colposcopy_test, ablation, excision])
msim = hpv.MultiSim(sim)
msim.run(n_runs=n)
msim.mean()

Loading location-specific demographic data for "tunisia"


Liste dans laquelle je sauvegarde les  données de chaque simulation sous forme de dictionnaire (liste de dictionnaires)

In [5]:
data_sce = [] 
vector_length = 101

results = msim.results
x=results['year']
       
vaccinated = list(results['n_vaccinated'])
vaccinated = [0]+[round(x) for x in vaccinated]
yearly_vaccinated = [vaccinated[i] - vaccinated[i-1] for i in range(1, len(vaccinated))]

yearly_pap = np.zeros(vector_length)
yearly_hpv_test = np.zeros(vector_length)
yearly_colposcopy = np.zeros(vector_length)
yearly_excision = np.zeros(vector_length)
yearly_ablation = np.zeros(vector_length)
for sim in msim.sims:
    yearly_pap += np.array(sim.get_intervention('pap primary').n_products_used)
    yearly_colposcopy += np.array(sim.get_intervention('colposcopy secondary').n_products_used)
    yearly_excision += np.array(sim.get_intervention('excision').n_products_used)
    yearly_ablation += np.array(sim.get_intervention('ablation').n_products_used)
yearly_pap = np.round(yearly_pap / n)
yearly_hpv_test = np.round(yearly_hpv_test / n)
yearly_colposcopy = np.round(yearly_colposcopy / n)
yearly_excision = np.round(yearly_excision / n)
yearly_ablation = np.round(yearly_ablation / n)

deaths = msim.results['cancer_deaths']
cancers = msim.results['cancers']
infections = msim.results['infections']

yearly_infections = infections
yearly_infections = [round(x) for x in yearly_infections]
coefficients = np.polyfit(x[:], yearly_infections, 5) 
yearly_infections_fitted = np.polyval(coefficients, x[:])
yearly_infections_fitted = [round(x) for x in yearly_infections_fitted]
yearly_infections_fitted = [0 if x < 0 else x for x in yearly_infections_fitted]

yearly_cancers = cancers
yearly_cancers = [round(x) for x in yearly_cancers]
coefficients = np.polyfit(x[:], yearly_cancers, 5) 
yearly_cancers_fitted = np.polyval(coefficients, x[:])
yearly_cancers_fitted = [round(x) for x in yearly_cancers_fitted]
yearly_cancers_fitted = [0 if x < 0 else x for x in yearly_cancers_fitted]

yearly_deaths = deaths
yearly_deaths = [round(x) for x in yearly_deaths]
coefficients = np.polyfit(x[:], yearly_deaths, 5) 
yearly_deaths_fitted = np.polyval(coefficients, x[:])
yearly_deaths_fitted = [round(x) for x in yearly_deaths_fitted]
yearly_deaths_fitted = [0 if x < 0 else x for x in yearly_deaths_fitted]

baseline_cancer =  list(yearly_cancers_fitted[35:])
baseline_death =  list(yearly_deaths_fitted[35:])

data_sce.append({'label': "baseline", 'vaccine_type': 'none', 'vaccinated_sex': 'none', 'hpv_screen_coverage': 0,
                                     'yearly_vaccinated':yearly_vaccinated[35:], 'total_vaccinated': np.sum(yearly_vaccinated[35:]),
                                     'yearly_pap':yearly_pap[35:], 'total_pap': np.sum(yearly_pap[35:]),
                                     'yearly_hpv_test':yearly_hpv_test[35:], 'total_hpv_test': np.sum(yearly_hpv_test[35:]),
                                     'yearly_colposcopy':yearly_colposcopy[35:], 'total_colposcopy': np.sum(yearly_colposcopy[35:]),
                                     'yearly_excision':yearly_excision[35:], 'total_excision': np.sum(yearly_excision[35:]),
                                     'yearly_ablation':yearly_ablation[35:],'total_ablation': np.sum(yearly_ablation[35:]),
                                     'yearly_infections': yearly_infections_fitted[35:], 'total_cancers': np.sum(yearly_infections_fitted[35:]),
                                     'yearly_cancers': yearly_cancers_fitted[35:], 'total_cancers': np.sum(yearly_cancers_fitted[35:]),
                                     'yearly_deaths': yearly_deaths_fitted[35:],'total_deaths': np.sum(yearly_deaths_fitted[35:]),
                                     'reduced_cancer_cases': round(sum(baseline_cancer)-sum(yearly_cancers_fitted[35:])), 
                                     'reduced_cancer_cases_%': round(100-np.sum(yearly_cancers_fitted[35:])/sum(baseline_cancer)*100, 2),
                                     'reduced_deaths_cases': round(sum(baseline_death)-sum(yearly_deaths_fitted[35:])), 
                                     'reduced_death_cases_%': round(100-np.sum(yearly_deaths_fitted[35:])/sum(baseline_death)*100, 2),
                                     'raw_cancers': cancers,'reduced_cancer_raw': 0
                            })

Fonction pour définir les vaccins (on peut faire avec protection croisée ou sans, par défaut de fais sans)

In [6]:
# %% Settings and filepaths
# Debug switch
debug = 0  # Run with smaller population sizes and in serial
do_shrink = True  # Do not keep people when running sims (saves memory)

# Save settings
do_save = True
save_plots = True



def make_vx(prod_name=None):
    dfvx = pd.read_csv(f'products_vx.csv')
    vxprods = dict()
    for name in dfvx.name.unique():
        vxprods[name] = hpv.vx(genotype_pars=dfvx[dfvx.name == name], imm_init=dict(dist='beta', par1=30, par2=2))
        vxprods[name+'2'] = hpv.vx(genotype_pars=dfvx[dfvx.name == name], imm_boost=1.2)  # 2nd dose
        vxprods[name+'3'] = hpv.vx(genotype_pars=dfvx[dfvx.name == name], imm_boost=1.1)  # 3rd dose
    if prod_name is not None:
       return vxprods[prod_name]
    else:
        return vxprods

Réalisation des simulations des autres scénarios

In [7]:
probs = [0, 0.15, 0.25, 0.33, 0.5, 0.7]
sex = [[0]]
vaccine = ['bivalent']
age = [[35, 40], [35, 45], [35, 60]]


k = 1
for a in age :
    for vac in vaccine:
        for s in sex:
            for prob in probs:
                print(k)
                k+=1
    
                vx_tun = make_vx(vac)
                routine_vx = hpv.routine_vx(
                    product=vx_tun,
                    age_range=[11, 12],
                    sex=s, 
                    prob=0.9, 
                    start_year=2025
                )
    
                def screen_eligible(sim):
                    import numpy as np
                    return np.isnan(sim.people.date_screened) | (sim.t > (sim.people.date_screened + 5 / sim['dt']))
    
                if (prob==0):
                    pap_test = hpv.routine_screening(
                        product='pap',
                        prob=0.01,
                        age_range = [35, 60],
                        start_year=2010,
                        label='pap primary'
                    )
                    
                else: 
                    pap_test = hpv.routine_screening(
                        product='pap',
                        prob=0.01,
                        age_range = [35, 60],
                        years=np.arange(2010, 2034),
                        label='pap primary'
                    )
                
    
                    gradual_hpv = hpv.routine_screening(
                        product='hpv', 
                        prob=np.linspace(0,prob,5), 
                        age_range = a,
                        years=np.arange(2030,2035),
                        label='hpv introduction',
                        eligibility=screen_eligible,
                    )
                    
                    hpv_test = hpv.routine_screening(
                        product='hpv',
                        prob=prob,
                        age_range = a,
                        start_year=2035,
                        label='hpv primary',
                        eligibility=screen_eligible,
                    )
    
                if prob==0:
                    to_colposcopy = lambda sim:list(set(sim.get_intervention('pap primary').outcomes['abnormal'].tolist()
                                                   ))
                else: 
                    to_colposcopy = lambda sim:list(set(sim.get_intervention('pap primary').outcomes['abnormal'].tolist()
                                                    + sim.get_intervention('hpv primary').outcomes['positive'].tolist()
                                                    + sim.get_intervention('hpv introduction').outcomes['positive'].tolist()
                                                   ))
                colposcopy_test = hpv.routine_triage(
                    product='colposcopy',
                    prob = 0.9,
                    eligibility=to_colposcopy,
                    label = 'colposcopy secondary'
                )
            
        
                hsils = lambda sim: sim.get_intervention('colposcopy secondary').outcomes['hsil']
                excision = hpv.treat_num(
                    product = 'excision',
                    prob = 0.9,
                    eligibility = hsils,
                    label = 'excision'
                )
            
                lsils = lambda sim: sim.get_intervention('colposcopy secondary').outcomes['lsil']
                ablation = hpv.treat_num(
                    product = 'ablation',
                    prob = 0.9,
                    eligibility = lsils,
                    label = 'ablation'
                )

                if (prob==0):
                    sim = hpv.Sim(params, interventions=[routine_vx, pap_test, colposcopy_test, ablation, excision])
                else : 
                    sim = hpv.Sim(params, interventions=[routine_vx, pap_test, gradual_hpv, hpv_test, colposcopy_test, ablation, excision])
                msim = hpv.MultiSim(sim) 
                msim.run(n_runs=10)


    
                label = ""
                if (s==[0]):
                    label = f'{vac} G + {round(prob*100)}% Screen. {a} '
                else:
                    label = f'{vac} BG + {round(prob*100)}% Screen. {a} '


                for sim in msim.sims:
                    results = sim.results
                    x = sim.results['year']
                
                    vaccinated = list(results['n_vaccinated'])
                    vaccinated = [0]+[round(x) for x in vaccinated]
                    yearly_vaccinated = [vaccinated[i] - vaccinated[i-1] for i in range(1, len(vaccinated))]
                
                    yearly_pap = np.zeros(vector_length)
                    yearly_hpv_test = np.zeros(vector_length)
                    yearly_colposcopy = np.zeros(vector_length)
                    yearly_excision = np.zeros(vector_length)
                    yearly_ablation = np.zeros(vector_length)
                    
                    yearly_pap += np.array(sim.get_intervention('pap primary').n_products_used)
                    if prob==0:
                        yearly_hpv_test = np.zeros(len(yearly_pap))
                    else:
                        yearly_hpv_test += np.array(sim.get_intervention('hpv introduction').n_products_used)+np.array(sim.get_intervention('hpv primary').n_products_used)
                    yearly_colposcopy += np.array(sim.get_intervention('colposcopy secondary').n_products_used)
                    yearly_excision += np.array(sim.get_intervention('excision').n_products_used)
                    yearly_ablation += np.array(sim.get_intervention('ablation').n_products_used)
                    
                    yearly_pap = np.round(yearly_pap)
                    yearly_hpv_test = np.round(yearly_hpv_test)
                    yearly_colposcopy = np.round(yearly_colposcopy)
                    yearly_excision = np.round(yearly_excision)
                    yearly_ablation = np.round(yearly_ablation)
                
                    deaths = sim.results['cancer_deaths']
                    cancers = sim.results['cancers']
                    infections = sim.results['infections']

                    yearly_infections = infections
                    yearly_infections = [round(x) for x in yearly_infections]
                    coefficients = np.polyfit(x[:], yearly_infections, 5) 
                    yearly_infections_fitted = np.polyval(coefficients, x[:])
                    yearly_infections_fitted = [round(x) for x in yearly_infections_fitted]
                    yearly_infections_fitted = [0 if x < 0 else x for x in yearly_infections_fitted]
                    
                    yearly_cancers = cancers
                    yearly_cancers = [round(x) for x in yearly_cancers]
                    coefficients = np.polyfit(x[:], yearly_cancers, 5) 
                    yearly_cancers_fitted = np.polyval(coefficients, x[:])
                    yearly_cancers_fitted = [round(x) for x in yearly_cancers_fitted]
                    yearly_cancers_fitted = [0 if x < 0 else x for x in yearly_cancers_fitted]
                    
                    yearly_deaths = deaths
                    yearly_deaths = [round(x) for x in yearly_deaths]
                    coefficients = np.polyfit(x[:], yearly_deaths, 5) 
                    yearly_deaths_fitted = np.polyval(coefficients, x[:])
                    yearly_deaths_fitted = [round(x) for x in yearly_deaths_fitted]
                    yearly_deaths_fitted = [0 if x < 0 else x for x in yearly_deaths_fitted]
                
    
    
                
                    data_sce.append({'label': label, 'vaccine_type': vac, 'vaccinated_sex': s, 'hpv_screen_coverage': prob,
                                         'yearly_vaccinated':yearly_vaccinated[35:], 'total_vaccinated': np.sum(yearly_vaccinated[35:]),
                                         'yearly_pap':yearly_pap[35:], 'total_pap': np.sum(yearly_pap[35:]),
                                         'yearly_hpv_test':yearly_hpv_test[35:], 'total_hpv_test': np.sum(yearly_hpv_test[35:]),
                                         'yearly_colposcopy':yearly_colposcopy[35:], 'total_colposcopy': np.sum(yearly_colposcopy[35:]),
                                         'yearly_excision':yearly_excision[35:], 'total_excision': np.sum(yearly_excision[35:]),
                                         'yearly_ablation':yearly_ablation[35:], 'total_ablation': np.sum(yearly_ablation[35:]),
                                         'yearly_infections': yearly_infections_fitted[35:], 'total_cancers': np.sum(yearly_infections_fitted[35:]),
                                         'yearly_cancers': yearly_cancers_fitted[35:], 'total_cancers': np.sum(yearly_cancers_fitted[35:]),
                                         'yearly_deaths': yearly_deaths_fitted[35:], 'total_deaths': np.sum(yearly_deaths_fitted[35:]),
                                         'reduced_cancer_cases': round(sum(baseline_cancer)-sum(yearly_cancers_fitted[35:])), 
                                         'reduced_cancer_cases_%': round(100-np.sum(yearly_cancers_fitted[35:])/sum(baseline_cancer)*100, 2),
                                         'reduced_deaths_cases': round(sum(baseline_death)-sum(yearly_deaths_fitted[35:])),
                                         'reduced_death_cases_%': round(100-np.sum(yearly_deaths_fitted[35:])/sum(baseline_death)*100, 2),
                                         'raw_cancers': cancers, 'reduced_cancer_raw':  round(100-np.sum(cancers[35:])/sum(baseline_cancer)*100, 2)
                                })
    
                print(label)


1
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 0% Screen. [35, 40] 
2
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 15% Screen. [35, 40] 
3
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 25% Screen. [35, 40] 
4
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 33% Screen. [35, 40] 
5
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 50% Screen. [35, 40] 
6
bivalent
quadrivalent
nonavalent
bivalent_without
quadrivalent_without
Loading location-specific demographic data for "tunisia"
bivalent G + 70% Screen. [35, 40] 
7
bivalent


In [52]:
scenarios_df = pd.DataFrame(data_sce)

In [53]:
scenarios_df.to_excel('scenarios_data.xlsx')